In [1]:
#!ls ~/trained_models/
from pathlib import Path
import pandas as pd
pd.set_option('display.max_colwidth', 20)   # JSON columns make things look weird in notebook without this

from datatype_recovery.models.dataset import load_dataset_from_path
from datatype_recovery.models.dataset.encoding import *


dataset_path = Path.home()/'datasets/tydamin_inmem'

# model_path = Path.home()/'trained_models/dragon_5hops_5heads_3linear__trainset_new_ep50.pt'

# eval_model_on_dataset(model_path, 'cuda:3', dataset_path)
dataset = load_dataset_from_path(dataset_path)

#df = dataset.read_vars_csv()
#df

In [ ]:
#df.groupby('TypeSeq_Debug').count().BinaryId
#df.TypeSeq_Debug.unique()

In [2]:
from datatype_recovery.models.training import split_train_test

train_indices, test_indices = split_train_test(len(dataset), batch_size=3)

Dataset size is 1,556,212
Test set has 155,619 samples (10.0%)
Train set has 1,400,592 samples (90.0%)
1 samples unused due to batch alignment (batch_size = 3)


In [3]:
from torch.utils.data import Subset
from tqdm import tqdm

test_set = [x for x in tqdm(Subset(dataset, list(test_indices)), desc='Loading test set')]
train_set = [x for x in tqdm(Subset(dataset, list(train_indices)), desc='Loading training set')]

Loading training set: 100%|██████████| 1400592/1400592 [02:56<00:00, 7931.16it/s]


In [6]:
torch.save(dataset.collate(test_set), Path.cwd()/'test2.pt')

In [8]:
out = torch.load(Path.cwd()/'test2.pt')
out

(Data(x=[5073407, 86], edge_index=[2, 9986088], edge_attr=[9986088, 26], varid=[155619], num_other_vars=[155619], y=[155619, 22]),
 {'x': tensor([      0,       5,      66,  ..., 5073184, 5073298, 5073407]),
  'edge_index': tensor([      0,       8,     130,  ..., 9985646, 9985872, 9986088]),
  'edge_attr': tensor([      0,       8,     130,  ..., 9985646, 9985872, 9986088]),
  'varid': tensor([     0,      1,      2,  ..., 155617, 155618, 155619]),
  'num_other_vars': tensor([     0,      1,      2,  ..., 155617, 155618, 155619]),
  'y': tensor([     0,      1,      2,  ..., 155617, 155618, 155619])})

In [14]:
out[0]

Data(x=[5073407, 86], edge_index=[2, 9986088], edge_attr=[9986088, 26], varid=[155619], num_other_vars=[155619], y=[155619, 22])

In [15]:
from torch_geometric.data import InMemoryDataset

class DefaultInMem(InMemoryDataset):
    '''
    Generic InMemoryDataset that saves/loads a list of Data objects
    '''
    def __init__(self, root, data_list, transform=None):
        self.data_list = data_list
        super().__init__(root, transform)
        self.load(self.processed_paths[0])

    @property
    def processed_file_names(self):
        return 'data.pt'

    def process(self):
        self.save(self.data_list, self.processed_paths[0])

In [16]:
tds = DefaultInMem('testset', test_set)
tds

Processing...
Done!


DefaultInMem(155619)

Data(x=[13, 86], edge_index=[2, 24], edge_attr=[24, 26], varid=[4], num_other_vars=[1], y=[1, 22])